In [6]:
import numpy
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv("Dataset/Titanic/train.csv")
print(data.head(n=10))

In [ ]:
#Drop features which not have much impact 
columns_to_drop = ["PassengerId","Name","Ticket","Cabin","Embarked"]
data_clean = data.drop(columns_to_drop,axis=1)

#Here we are transforming "Sex" columns which has values Male and Female, into 1 and 0 respectively so that the algorithm can understand easily.
le = LabelEncoder()
data_clean["Sex"] = le.fit_transform(data_clean["Sex"])

In [9]:
data_clean

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000
887,1,1,0,19.0,0,0,30.0000
888,0,3,0,NaN,1,2,23.4500
889,1,1,1,26.0,0,0,30.0000


In [11]:
data_clean = data_clean.fillna(data_clean["Age"].mean())

In [12]:
input_cols = ['Pclass',"Sex","Age","SibSp","Parch","Fare"]
output_cols = ["Survived"]

X = data_clean[input_cols]
Y = data_clean[output_cols]

print(X.shape,Y.shape)
print(type(X))

(891, 6) (891, 1)
<class 'pandas.core.frame.DataFrame'>
